In [ ]:
# Copy train images to  "target_folder"
from pathlib import Path
import itertools
import re
import random
import shutil

# Specify the directories
raw_data_folder = Path('/Users/vkluzner/OneDrive/NeuralMorphology/SimulationsNew_Tif3334/images/')
processed_folder = Path('/Users/vkluzner/OneDrive/NeuralMorphology/SimulationsNew_Tif3334/neuro-morpho/')

# Ensure the destination folders exist
for dir_path, subdir in itertools.product([Path('train'), Path('val'), Path('test')], ["imgs", "lbls"]):
    (processed_folder / dir_path / subdir).mkdir(parents=True, exist_ok=True)

# List all image files in the raw_data_folder
image_files = sorted(list(Path(raw_data_folder).glob("Realistic-SBR-*")))

# Loop through files and organize them
for image_name in image_files:
    label_name = raw_data_folder / re.sub(r"Realistic-SBR-.", "Skeleton", image_name.name)
    if random.random() <= 0.6: # train data
        shutil.copy(image_name, processed_folder / 'train' / 'imgs')
        new_label_name = processed_folder / 'train' / 'lbls' / image_name.name.replace("Realistic-SBR-", "Skeleton-")
    elif random.random() <= 0.8: # validation data
        shutil.copy(image_name, processed_folder / 'val' / 'imgs')
        new_label_name = processed_folder / 'val' / 'lbls' / image_name.name.replace("Realistic-SBR-", "Skeleton-")
    else: # test data
        shutil.copy(image_name, processed_folder / 'test' / 'imgs')
        new_label_name = processed_folder / 'test' / 'lbls' / image_name.name.replace("Realistic-SBR-", "Skeleton-")
    shutil.copy(label_name, new_label_name)
